In [ ]:
OLD_EXPERIMENT = '1018_RMSPROP_lr=8e-06_l2=1e-02_pytorch_init_casia_gh'

In [ ]:
!nvidia-smi

Thu Nov  4 07:58:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive

MOUNT_POINT = '/content/gdrive' 
drive.mount(MOUNT_POINT,  force_remount=True)
MOUNT_POINT += "/My Drive"

Mounted at /content/gdrive


In [ ]:
import datetime
REF_EXPERIMENT = OLD_EXPERIMENT + '_' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [ ]:
REF_EXPERIMENT

'1018_RMSPROP_lr=8e-06_l2=1e-02_pytorch_init_casia_gh_20211104-075833'

In [ ]:
!pip install icecream
from icecream import ic

import os
import json
import matplotlib.pyplot as plt
!pip install tensorflow-io

In [ ]:
#folders
PWD = os.path.join(MOUNT_POINT, "4.2/visiope/PROJECT")

In [ ]:
#export result
EXPORT_ROOT = os.path.join(PWD, 'results')
if not os.path.isdir(EXPORT_ROOT):
  os.makedirs(EXPORT_ROOT)
EXPORT_CURRENT_EXPERIMENT = os.path.join(EXPORT_ROOT, REF_EXPERIMENT)

In [ ]:
SETUPS = os.path.join(EXPORT_CURRENT_EXPERIMENT, 'setups')
MODELS = os.path.join(EXPORT_CURRENT_EXPERIMENT, 'models')
LOG_TRAINING_TESTING = os.path.join(EXPORT_CURRENT_EXPERIMENT, 'log_training_testing')
BEST_WEIGHTS = os.path.join(EXPORT_CURRENT_EXPERIMENT, 'weights')

LOGS = os.path.join(EXPORT_CURRENT_EXPERIMENT, 'logs')

CHECKPOINT = os.path.join(EXPORT_CURRENT_EXPERIMENT, 'checkpoint')

In [ ]:
if os.path.isdir(EXPORT_CURRENT_EXPERIMENT):
  print("Error! Current experiment has been already performed, change REF_EXPERIMENT to run a new experiment")
  raise KeyboardInterrupt
else:
  os.makedirs(SETUPS)
  os.makedirs(MODELS)
  os.makedirs(LOG_TRAINING_TESTING)
  os.makedirs(BEST_WEIGHTS)
  os.makedirs(LOGS)
  os.makedirs(CHECKPOINT)

In [ ]:

OLD_EXPORT_ROOT = os.path.join(PWD, 'results')
OLD_EXPORT_CURRENT_EXPERIMENT = os.path.join(OLD_EXPORT_ROOT, OLD_EXPERIMENT)
OLD_CHECKPOINT = os.path.join(OLD_EXPORT_CURRENT_EXPERIMENT, 'checkpoint')
OLD_SETUPS = os.path.join(OLD_EXPORT_CURRENT_EXPERIMENT, 'setups')

# get last checkpoint
checks = sorted(os.listdir(OLD_CHECKPOINT), reverse=True)
FOLDER_OLD_CHECKPOINT = os.path.join(OLD_CHECKPOINT, checks[0])

In [ ]:
checks

['epoch=0199']

In [ ]:
def export_setting(name_setting, setting):
  #export setting
  filename = os.path.join(SETUPS, name_setting)
  with open(filename, "w") as outfile:
      json.dump(setting, outfile, indent=4)
      outfile.close()

def load_setting(name_setting):
    #import setting
    filename = os.path.join(OLD_SETUPS, name_setting)
    setting = {}
    with open(filename, "r") as infile:
        setting=json.load(infile)
        infile.close()
    return setting

In [ ]:
SETTING = load_setting('SETTING.json')
SETTING

{'ADAM_LEARNING_RATE': 8e-06,
 'BATCH_SIZE': 10,
 'BIAS_INITIALIZER': 'pytorch',
 'DATASET': 'casia_v2_github_gt',
 'EPOCHS': 200,
 'IMG_HEIGHT': 135,
 'IMG_WIDTH': 240,
 'KERNEL_INITIALIZER': 'pytorch',
 'LEARNING_RATE': 8e-06,
 'LOSS': 'binary_cross_entropy',
 'MODEL': 'RRUNET',
 'NORMALIZATION': 'gn',
 'OPTIMIZER': 'rmsprop',
 'SEED_RANDOM': 20201205,
 'SGD_LEARNING_RATE': 8e-06,
 'SGD_MOMENTUM': 0.9,
 'SGD_NESTEROV': True,
 'SGD_WEIGHT_DECAY': 0.01}

In [ ]:
#SETTING['EPOCHS'] = 100
SETTING['LEARNING_RATE'] = 1E-06

In [ ]:
SETTING

{'ADAM_LEARNING_RATE': 8e-06,
 'BATCH_SIZE': 10,
 'BIAS_INITIALIZER': 'pytorch',
 'DATASET': 'casia_v2_github_gt',
 'EPOCHS': 200,
 'IMG_HEIGHT': 135,
 'IMG_WIDTH': 240,
 'KERNEL_INITIALIZER': 'pytorch',
 'LEARNING_RATE': 1e-06,
 'LOSS': 'binary_cross_entropy',
 'MODEL': 'RRUNET',
 'NORMALIZATION': 'gn',
 'OPTIMIZER': 'rmsprop',
 'SEED_RANDOM': 20201205,
 'SGD_LEARNING_RATE': 8e-06,
 'SGD_MOMENTUM': 0.9,
 'SGD_NESTEROV': True,
 'SGD_WEIGHT_DECAY': 0.01}

In [ ]:
#input dataset
DATASET_ROOT_IMAGE = os.path.join(PWD, SETTING['DATASET'], 'images')

## TRAIN
TRAINING_SET_IMAGE = os.path.join(DATASET_ROOT_IMAGE, 'train/images')

## VALIDATION
VALIDATION_SET_IMAGE = os.path.join(DATASET_ROOT_IMAGE, 'val/images')

## TEST
TEST_SET_IMAGE = os.path.join(DATASET_ROOT_IMAGE, 'test/images')

In [ ]:
export_setting('SETTING.json', SETTING)

# RRU-Net

In [ ]:
!pip install tensorflow-addons

In [ ]:
import numpy as np
import tensorflow as tf

import keras

print("Tensorflow version %s" %tf.__version__)
print("Keras version %s" %keras.__version__)

device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
  print('Found GPU at: {}'.format(device_name))

Tensorflow version 2.6.0
Keras version 2.6.0


In [ ]:
import sys
sys.path.append(PWD)

import RRU_net
import Dataset
import Utils
import Evaluation
import Losses

In [ ]:
#set seeds
from numpy.random import seed 
seed(SETTING['SEED_RANDOM'])

tf.random.set_seed(SETTING['SEED_RANDOM'])

### RRU-Net Model

In [ ]:
if SETTING['MODEL'] == 'RRUNET':
  model = RRU_net.RRU_net(l2_penalty=SETTING['SGD_WEIGHT_DECAY'],
                            normalization=SETTING['NORMALIZATION'],
                            kernel_initializer=SETTING['KERNEL_INITIALIZER'],
                            bias_initializer=SETTING['BIAS_INITIALIZER'],
                            seed_initializer=SETTING['SEED_RANDOM']                          
                          )
  input_x = tf.random.uniform((1, 6, 14, 3))
  y_hat = model(input_x)
  model.summary()
else:
  model = None

Model: "rru_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
down_0 (RRU_first_down)      multiple                  10528     
_________________________________________________________________
down_1 (RRU_down)            multiple                  59904     
_________________________________________________________________
down_2 (RRU_down)            multiple                  238592    
_________________________________________________________________
down_3 (RRU_down)            multiple                  952320    
_________________________________________________________________
down_4 (RRU_down)            multiple                  1312768   
_________________________________________________________________
up_1 (RRU_up)                multiple                  1132288   
_________________________________________________________________
up_2 (RRU_up)                multiple                  2835

In [ ]:
#model = RRU_net.RRU_net(l2_penalty=SETTING['SGD_WEIGHT_DECAY'], normalization=SETTING['NORMALIZATION'])
#input_x = tf.random.uniform((1, 6, 14, 3))
#y_hat = model(input_x)
#model.summary()

# Dataset

In [ ]:
dataset_builder = Dataset.Dataset(training_image_folder=TRAINING_SET_IMAGE,
                                  validation_image_folder=VALIDATION_SET_IMAGE, 
                                  batch_size=SETTING['BATCH_SIZE'], 
                                  random_seed=SETTING['SEED_RANDOM'],
                                  img_height=SETTING['IMG_HEIGHT'],
                                  img_width=SETTING['IMG_WIDTH'])

None
Tensor("case/cond/Identity:0", shape=(), dtype=string)
None
Tensor("case/cond/Identity:0", shape=(), dtype=string)


In [ ]:
dataset = dataset_builder.dataset

TRAINSET_SIZE = dataset_builder.training_set_size
VALSET_SIZE = dataset_builder.validation_set_size

In [ ]:
print(dataset['train'])
print(dataset['val'])

<PrefetchDataset shapes: ((None, 135, 240, None), (None, 135, 240, 1)), types: (tf.float32, tf.uint8)>
<PrefetchDataset shapes: ((None, 135, 240, None), (None, 135, 240, 1)), types: (tf.float32, tf.uint8)>


# EVALUATION

In [ ]:
def eval_val():
  return Evaluation.evaluation(model, dataset['val'], VALSET_SIZE, SETTING['BATCH_SIZE'])

## LOSS

In [ ]:
def loss_example(image, true_mask):
  pred = model.predict(image)
  return Losses.binary_cross_entropy_flatten(true_mask, pred)


for image, mask in dataset['train'].take(1):
  sample_image, sample_mask = image, mask

ic(loss_example(sample_image, sample_mask));

ic| loss_example(sample_image, sample_mask): <tf.Tensor: shape=(), dtype=float32, numpy=0.73430836>


# TRANING

In [ ]:
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.metrics import Recall, Accuracy   

if SETTING['OPTIMIZER'] == 'SGD':
  opt = tf.keras.optimizers.SGD(
                                learning_rate=SETTING['SGD_LEARNING_RATE'], 
                                momentum=SETTING['SGD_MOMENTUM'], 
                                nesterov=SETTING['SGD_NESTEROV'], name="SGD")
elif SETTING['OPTIMIZER'] == 'adam':
  opt = tf.keras.optimizers.Adam(learning_rate=SETTING['ADAM_LEARNING_RATE'])
  print("adam opt")
elif SETTING['OPTIMIZER'] == 'rmsprop':
  opt = tf.keras.optimizers.RMSprop(
    learning_rate=SETTING['LEARNING_RATE'], rho=0.9, momentum=0.0, epsilon=1e-07, centered=False, name='RMSprop')
  print("rmsprop opt")
else:
  opt = SETTING['OPTIMIZER']
  print("warning unexpeted opt: " + opt)

if SETTING['LOSS'] == 'binary_cross_entropy':
  loss = Losses.binary_cross_entropy_flatten
elif SETTING['LOSS'] == 'dice_loss':
  loss = Losses.dice_loss_flatten
else:
  raise SystemError('Unknown loss function!')

model.compile(optimizer=opt, loss=loss)

rmsprop opt


## LOAD PREVIOUS STATUS

In [ ]:
#eval_val()

In [ ]:
model_history = model.fit(dataset['train'], 
                          epochs=1,
                          steps_per_epoch=1,
                          validation_steps=1,
                          validation_data=dataset['val'],
                          verbose=1, 
                          callbacks=[                            
                                     ]
                          )

1/1 [==============================] - 20s 20s/step - loss: 16.7074 - val_loss: 16.7063


In [ ]:
FOLDER_OLD_CHECKPOINT

'/content/gdrive/My Drive/4.2/visiope/PROJECT/results/1018_RMSPROP_lr=8e-06_l2=1e-02_pytorch_init_casia_gh/checkpoint/epoch=0199'

In [ ]:
Utils.load_weights_and_optimizer(model, FOLDER_OLD_CHECKPOINT)

In [ ]:
#eval_val()

In [ ]:
Utils.save_weights(model, os.path.join(BEST_WEIGHTS, 'PRE_TRAINING'))

In [ ]:
import datetime

logdir = os.path.join(LOGS, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

file_writer = tf.summary.create_file_writer(os.path.join(logdir, 'metrics'))
file_writer.set_as_default() 

In [ ]:
EPOCHS = SETTING['EPOCHS']

STEPS_PER_EPOCH = TRAINSET_SIZE // SETTING['BATCH_SIZE']
VALIDATION_STEPS = VALSET_SIZE // SETTING['BATCH_SIZE']

In [ ]:
import os

In [ ]:
model_history = model.fit(dataset['train'], 
                          epochs=EPOCHS,
                          steps_per_epoch=STEPS_PER_EPOCH,
                          validation_steps=VALIDATION_STEPS,
                          validation_data=dataset['val'],
                          verbose=1, 
                          callbacks=[
                                     Evaluation.evaluation_save_best_weigths(
                                             _valid_data=dataset['val'], 
                                             _val_size=VALSET_SIZE, 
                                             _batch_size=SETTING['BATCH_SIZE'],
                                             _SETTING=SETTING,
                                             _BEST_WEIGHTS=BEST_WEIGHTS, 
                                             _CHECKPOINT=CHECKPOINT
                                             #,
                                             #delete_old_checkpoint=True
                                             ),
                                     tensorboard_callback                                     
                                     ]
                          )

Epoch 1/200
435/435 [==============================] - 151s 345ms/step - loss: 0.1502 - val_loss: 0.5130
 — val_f1: 0.331315 — val_precision: 0.749215 — val_recall: 0.212684 - val_dice_loss: 0.304053
Epoch 2/200
435/435 [==============================] - 142s 326ms/step - loss: 0.1473 - val_loss: 0.5151
 — val_f1: 0.330106 — val_precision: 0.750062 — val_recall: 0.211621 - val_dice_loss: 0.282343
Epoch 3/200
435/435 [==============================] - 142s 327ms/step - loss: 0.1458 - val_loss: 0.5148
 — val_f1: 0.329481 — val_precision: 0.757735 — val_recall: 0.210507 - val_dice_loss: 0.291417
Epoch 4/200
435/435 [==============================] - 142s 327ms/step - loss: 0.1445 - val_loss: 0.5136
 — val_f1: 0.331927 — val_precision: 0.777212 — val_recall: 0.211025 - val_dice_loss: 0.272113
Epoch 5/200
435/435 [==============================] - 142s 327ms/step - loss: 0.1433 - val_loss: 0.5142
 — val_f1: 0.328956 — val_precision: 0.761717 — val_recall: 0.209775 - val_dice_loss: 0.290212


In [ ]:
drive.flush_and_unmount() 